In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
data = pd.read_csv("df_criado_final.csv")
data.shape

(50390, 23)

In [3]:
data.head()

,Escolaridade,Renda Mensal Informal,Dependentes,Estado Civil,Idade,Conta Poupanca,Conta Salario,Quant Adiantamento Deposito,Qtd Fonte Renda,Cheque Sem Fundo,...,Valor Emprestimo,Multa,Juros,Valor Emprestimo Atualizado,PAGO,Controle,ID,Genero,Data,Estado
0,NaN,NaN,S,2,36,S,N,NaN,NaN,N,...,8174.32,263.25,5844.98,14282.55,1,1,1328587.0,1,2020-01-01,57
1,1.0,1894.5078,N,2,39,S,N,2.0,1.0,S,...,41775.63,1607.29,7652.26,51035.18,1,1,1424254.0,1,2020-01-01,59
2,1.0,NaN,S,2,26,S,N,NaN,1.0,S,...,522940.27,24860.40,319315.62,867116.29,1,1,1438519.0,0,2020-01-01,56
3,NaN,NaN,N,8,23,N,N,NaN,NaN,N,...,1593.00,21472.36,2967.48,26032.84,1,1,1426138.0,1,2020-01-01,56
4,NaN,NaN,S,2,33,S,N,NaN,NaN,N,...,2078.97,19.44,46.35,2144.76,1,1,1474504.0,0,2020-01-01,59


In [4]:
#Retira colunas vazias
data = data.drop(columns=['Valor Conta Corrente', 'Valor Conta Poupanca'])

In [5]:
#Função que verifica se todos os valores de uma coluna são iguais, retorno booleano
def is_unique(s):
    a = s.to_numpy() 
    return (a[0] == a).all()

colunas_valores_iguais = []

#Remove colunas redundantes
for i in range(21):
    #print(data.columns[i] + ":\n" + str(is_unique(data[data.columns[i]])))
    if(is_unique(data[data.columns[i]])):
        colunas_valores_iguais.append(data.columns[i])
    
data = data.drop(columns = colunas_valores_iguais)
data.shape

(50390, 19)

In [6]:
data.columns

Index(['Escolaridade', 'Renda Mensal Informal', 'Dependentes', 'Estado Civil',
       'Idade', 'Conta Poupanca', 'Conta Salario',
       'Quant Adiantamento Deposito', 'Qtd Fonte Renda', 'Cheque Sem Fundo',
       'Conta Conjunta', 'Valor Emprestimo', 'Multa', 'Juros',
       'Valor Emprestimo Atualizado', 'PAGO', 'ID', 'Genero', 'Estado'],
      dtype='object')

In [7]:
#Cria a coluna Valor Real Pago e adiciona valores

data['Valor Real Pago %'] = 0
data['Multa %'] = 0

def completa_coluna_valor_real(df):
    for index, row in df.iterrows():
        valor_real = 0
        multa_porcentagem = 0
        
        multa_porcentagem = float((df.loc[index, 'Multa'] * 100) / df.loc[index, 'Valor Emprestimo'])
        
        valor_real = float(((df.loc[index, 'Valor Emprestimo Atualizado'] - df.loc[index, 'Valor Emprestimo'])/
                      df.loc[index, 'Valor Emprestimo']) * 100 ) 
        
        df.loc[index,'Valor Real Pago %'] =  valor_real
        df.loc[index,'Multa %'] =  multa_porcentagem
        
completa_coluna_valor_real(data)  
data.head()

,Escolaridade,Renda Mensal Informal,Dependentes,Estado Civil,Idade,Conta Poupanca,Conta Salario,Quant Adiantamento Deposito,Qtd Fonte Renda,Cheque Sem Fundo,...,Valor Emprestimo,Multa,Juros,Valor Emprestimo Atualizado,PAGO,ID,Genero,Estado,Valor Real Pago %,Multa %
0,NaN,NaN,S,2,36,S,N,NaN,NaN,N,...,8174.32,263.25,5844.98,14282.55,1,1328587.0,1,57,74.724625,3.220451
1,1.0,1894.5078,N,2,39,S,N,2.0,1.0,S,...,41775.63,1607.29,7652.26,51035.18,1,1424254.0,1,59,22.164956,3.847434
2,1.0,NaN,S,2,26,S,N,NaN,1.0,S,...,522940.27,24860.40,319315.62,867116.29,1,1438519.0,0,56,65.815551,4.753965
3,NaN,NaN,N,8,23,N,N,NaN,NaN,N,...,1593.00,21472.36,2967.48,26032.84,1,1426138.0,1,56,1534.202134,1347.919648
4,NaN,NaN,S,2,33,S,N,NaN,NaN,N,...,2078.97,19.44,46.35,2144.76,1,1474504.0,0,59,3.164548,0.935078


In [8]:
#Separa DataSet em quem pagou e não pagou emprestimos passados

bons_pagadores = data.query('PAGO == 1')
mals_pagadores = data.query('PAGO == 0')

print(bons_pagadores.shape, mals_pagadores.shape,(bons_pagadores.shape[0] + mals_pagadores.shape[0]) )

(39853, 21) (10537, 21) 50390


In [9]:
#Relacionamento entre Dependentes, Estado Civil e Conta Conjunta
# Sem dependentes: Apresentou maior número de mals pagadores
#'Casado(a) com comunhão parcial de bens': Dentre todas a análises individuais, 
#foi o que apresentou um maior número de mals pagadores e tinha um número expressivo de individuos no banco.
#Sem conta conjunta: Apresentou maior percentual dentroda análise individual

Relacionamento_Dependentes_EstadoCivil_ContaConjunta_Bons = bons_pagadores.query(
    'Dependentes == "N" & `Estado Civil` == 4 & `Conta Conjunta` == "N"')

Relacionamento_Dependentes_EstadoCivil_ContaConjunta_Mals = mals_pagadores.query(
    'Dependentes == "N" & `Estado Civil` == 4 & `Conta Conjunta` == "N"')

print("Quantidade de dados: {}\n".format(Relacionamento_Dependentes_EstadoCivil_ContaConjunta_Bons.shape[0] + 
 Relacionamento_Dependentes_EstadoCivil_ContaConjunta_Mals.shape[0]))

porcentagem_Rel_Dep_EC_CC_Bons = (Relacionamento_Dependentes_EstadoCivil_ContaConjunta_Bons.shape[0]/
(Relacionamento_Dependentes_EstadoCivil_ContaConjunta_Bons.shape[0] + 
 Relacionamento_Dependentes_EstadoCivil_ContaConjunta_Mals.shape[0])) * 100

print('Relacionamento entre SEM dependentes | Casado Comunhão Parcial | Sem Conta Conjunta\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_Rel_Dep_EC_CC_Bons, 
                                                     (100 - porcentagem_Rel_Dep_EC_CC_Bons)))


Quantidade de dados: 11844

Relacionamento entre SEM dependentes | Casado Comunhão Parcial | Sem Conta Conjunta

Bons pagadores: 59.616683552853765%
Mals pagadores: 40.383316447146235%


In [10]:
Relacionamento_Dependentes_ContaConjunta_Bons = bons_pagadores.query(
    'Dependentes == "N" & `Conta Conjunta` == "N"')

Relacionamento_Dependentes_ContaConjunta_Mals = mals_pagadores.query(
    'Dependentes == "N" & `Conta Conjunta` == "N"')

print("Quantidade de dados: {}\n".format(Relacionamento_Dependentes_ContaConjunta_Bons.shape[0] + 
 Relacionamento_Dependentes_ContaConjunta_Mals.shape[0]))


porcentagem_Rel_Dep_CC_Bons = (Relacionamento_Dependentes_ContaConjunta_Bons.shape[0]/
(Relacionamento_Dependentes_ContaConjunta_Bons.shape[0] + 
 Relacionamento_Dependentes_ContaConjunta_Mals.shape[0])) * 100

print('Relacionamento entre SEM dependentes | Sem Conta Conjunta\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_Rel_Dep_CC_Bons, 
                                                     (100 - porcentagem_Rel_Dep_CC_Bons)))

Quantidade de dados: 31094

Relacionamento entre SEM dependentes | Sem Conta Conjunta

Bons pagadores: 72.43841255547694%
Mals pagadores: 27.561587444523056%


In [11]:
#Relacionamento casado parcial e SEM depedentes
Relacionamento_Dependentes_EstadoCivil_Bons = bons_pagadores.query(
    'Dependentes == "N" & `Estado Civil` == 4')

Relacionamento_Dependentes_EstadoCivil_Mals = mals_pagadores.query(
    'Dependentes == "N" & `Estado Civil` == 4')

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_Dependentes_EstadoCivil_Bons.shape[0] + 
 Relacionamento_Dependentes_EstadoCivil_Mals.shape[0]))


porcentagem_Rel_Dep_EC_Bons = (Relacionamento_Dependentes_EstadoCivil_Bons.shape[0]/
(Relacionamento_Dependentes_EstadoCivil_Bons.shape[0] + 
 Relacionamento_Dependentes_EstadoCivil_Mals.shape[0])) * 100

print('Relacionamento entre SEM dependentes | Casado Comunhão Parcial\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_Rel_Dep_EC_Bons, 
                                                     (100 - porcentagem_Rel_Dep_EC_Bons)))


Quantidade de dados: 11915

Relacionamento entre SEM dependentes | Casado Comunhão Parcial

Bons pagadores: 59.64750314729332%
Mals pagadores: 40.35249685270668%


In [12]:
salarioMinimo = 1045
print(salarioMinimo * 50)

52250


In [13]:
#Relacionamento entre Renda Informal e Idade

Relacionamento_RendaInformal_Idade_Bons = bons_pagadores.query(
    '`Renda Mensal Informal` >= 52250 & `Idade` >= 60')

Relacionamento_RendaInformal_Idade_Mals = mals_pagadores.query(
    '`Renda Mensal Informal` >= 52250 & `Idade` >= 60')

print("Quantidade de dados: {}\n".format(Relacionamento_RendaInformal_Idade_Bons.shape[0] + 
 Relacionamento_RendaInformal_Idade_Mals.shape[0]))

porcentagem_RendaInformal_Idade_Bons = (Relacionamento_RendaInformal_Idade_Bons.shape[0]/
(Relacionamento_RendaInformal_Idade_Bons.shape[0] + 
 Relacionamento_RendaInformal_Idade_Mals.shape[0])) * 100

print('Relacionamento entre Renda Informal Maior que 50 salários Mínimos | Idade Maior que 60 anos\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_RendaInformal_Idade_Bons, 
                                                     (100 - porcentagem_RendaInformal_Idade_Bons)))

Quantidade de dados: 84

Relacionamento entre Renda Informal Maior que 50 salários Mínimos | Idade Maior que 60 anos

Bons pagadores: 40.476190476190474%
Mals pagadores: 59.523809523809526%


In [14]:
#Relacionamento casado parcial ,SEM depedentes e valor real pago
Relacionamento_Dependentes_EstadoCivil_VRP_Bons = bons_pagadores.query(
    'Dependentes == "N" & `Estado Civil` == 4 & `Valor Real Pago %` > 80')

Relacionamento_Dependentes_EstadoCivil_VRP_Mals = mals_pagadores.query(
    'Dependentes == "N" & `Estado Civil` == 4 & `Valor Real Pago %` > 80')

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_Dependentes_EstadoCivil_VRP_Bons.shape[0] + 
 Relacionamento_Dependentes_EstadoCivil_VRP_Mals.shape[0]))


porcentagem_Rel_Dep_EC_VRP_Bons = (Relacionamento_Dependentes_EstadoCivil_VRP_Bons.shape[0]/
(Relacionamento_Dependentes_EstadoCivil_VRP_Bons.shape[0] + 
 Relacionamento_Dependentes_EstadoCivil_VRP_Mals.shape[0])) * 100

print('Relacionamento entre SEM dependentes | Casado Comunhão Parcial | Valor Real Pago > 80%\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_Rel_Dep_EC_VRP_Bons, 
                                                     (100 - porcentagem_Rel_Dep_EC_VRP_Bons)))



Quantidade de dados: 6661

Relacionamento entre SEM dependentes | Casado Comunhão Parcial | Valor Real Pago > 80%

Bons pagadores: 44.58789971475755%
Mals pagadores: 55.41210028524245%


In [15]:
#Análise para bons pagadores

    #Cheque sem fundo: 'S' [90% bons]
    #Estado Civil: 2(Solteiro) [89% bons]  
    #Escolaridade:1(Ensino Médio) [90% bons] 
    #Dependentes: 'S' [90% bons]
    #Valor Real: <60% [~90% bons]
    #Multa %: <10% [94,5% bons]

Relacionamento_chequeSemFundo_EC_Escol_Dep_VR_Multa_Bons = bons_pagadores.query(
    '`Cheque Sem Fundo` == "S" & `Estado Civil` == 2 & Escolaridade == 1 & \
    Dependentes == "S" & `Valor Real Pago %` <= 60 & `Multa %` <= 20' )

Relacionamento_chequeSemFundo_EC_Escol_Dep_VR_Multa_Mals = mals_pagadores.query(
    '`Cheque Sem Fundo` == "S" & `Estado Civil` == 2 & Escolaridade == 1 & \
    Dependentes == "S" & `Valor Real Pago %` <= 60 & `Multa %` <= 20' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_chequeSemFundo_EC_Escol_Dep_VR_Multa_Bons.shape[0] + 
 Relacionamento_chequeSemFundo_EC_Escol_Dep_VR_Multa_Mals.shape[0]))


porcentagem_chequeSemFundo_EC_Escol_Dep_VR_Multa_Bons = (Relacionamento_chequeSemFundo_EC_Escol_Dep_VR_Multa_Bons.shape[0]/
(Relacionamento_chequeSemFundo_EC_Escol_Dep_VR_Multa_Bons.shape[0] + 
 Relacionamento_chequeSemFundo_EC_Escol_Dep_VR_Multa_Mals.shape[0])) * 100

print('Relacionamento entre COM Cheque Sem Fundo | Solteiro | Ensino Médio | COM Dependentes \
| Valor Real <=60% | Multa <= 20%\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_chequeSemFundo_EC_Escol_Dep_VR_Multa_Bons, 
                                                     (100 - porcentagem_chequeSemFundo_EC_Escol_Dep_VR_Multa_Bons)))


Quantidade de dados: 1100

Relacionamento entre COM Cheque Sem Fundo | Solteiro | Ensino Médio | COM Dependentes | Valor Real <=60% | Multa <= 20%

Bons pagadores: 95.36363636363636%
Mals pagadores: 4.63636363636364%


In [16]:

Relacionamento_EC_Escol_Dep_Bons = bons_pagadores.query(
    '`Estado Civil` == 2 & Escolaridade == 1 & Dependentes == "S"' )

Relacionamento_EC_Escol_Dep_Mals = mals_pagadores.query(
    '`Estado Civil` == 2 & Escolaridade == 1 & Dependentes == "S"' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_EC_Escol_Dep_Bons.shape[0] + 
 Relacionamento_EC_Escol_Dep_Mals.shape[0]))


porcentagem_EC_Escol_Dep_Bons = (Relacionamento_EC_Escol_Dep_Bons.shape[0]/
(Relacionamento_EC_Escol_Dep_Bons.shape[0] + Relacionamento_EC_Escol_Dep_Mals.shape[0])) * 100

print('Relacionamento entre Estado Civil: Solteiro | Escolaridade: Ensino Médio | COM Dependentes\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_EC_Escol_Dep_Bons, 
                                                     (100 - porcentagem_EC_Escol_Dep_Bons)))


Quantidade de dados: 6534

Relacionamento entre Estado Civil: Solteiro | Escolaridade: Ensino Médio | COM Dependentes

Bons pagadores: 92.30180593816958%
Mals pagadores: 7.698194061830421%


In [17]:
Relacionamento_EC_Dep_Bons = bons_pagadores.query(
    '`Estado Civil` == 2 & Dependentes == "S"' )

Relacionamento_EC_Dep_Mals = mals_pagadores.query(
    '`Estado Civil` == 2 & Dependentes == "S"' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_EC_Dep_Bons.shape[0] + 
 Relacionamento_EC_Dep_Mals.shape[0]))


porcentagem_EC_Dep_Bons = (Relacionamento_EC_Dep_Bons.shape[0]/
(Relacionamento_EC_Dep_Bons.shape[0] + Relacionamento_EC_Dep_Mals.shape[0])) * 100

print('Relacionamento entre Estado Civil: Solteiro | COM Dependentes\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_EC_Dep_Bons, 
                                                     (100 - porcentagem_EC_Dep_Bons)))


Quantidade de dados: 15785

Relacionamento entre Estado Civil: Solteiro | COM Dependentes

Bons pagadores: 92.09375989863796%
Mals pagadores: 7.9062401013620445%


In [18]:
Relacionamento_EC_Escol_Bons = bons_pagadores.query(
    '`Estado Civil` == 2 & Escolaridade == 1' )

Relacionamento_EC_Escol_Mals = mals_pagadores.query(
    '`Estado Civil` == 2 & Escolaridade == 1' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_EC_Escol_Bons.shape[0] + 
 Relacionamento_EC_Escol_Mals.shape[0]))


porcentagem_EC_Escol_Bons = (Relacionamento_EC_Escol_Bons.shape[0]/
(Relacionamento_EC_Escol_Bons.shape[0] + Relacionamento_EC_Escol_Mals.shape[0])) * 100

print('Relacionamento entre Estado Civil: Solteiro | Escolaridade: Ensino Médio\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_EC_Escol_Bons, 
                                                     (100 - porcentagem_EC_Escol_Bons)))


Quantidade de dados: 12358

Relacionamento entre Estado Civil: Solteiro | Escolaridade: Ensino Médio

Bons pagadores: 90.21686357015697%
Mals pagadores: 9.783136429843026%


In [19]:
Relacionamento_Escol_Dep_Bons = bons_pagadores.query(
    'Escolaridade == 1 & Dependentes == "S"' )

Relacionamento_Escol_Dep_Mals = mals_pagadores.query(
    'Escolaridade == 1 & Dependentes == "S"' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_Escol_Dep_Bons.shape[0] + 
 Relacionamento_Escol_Dep_Mals.shape[0]))


porcentagem_Escol_Dep_Bons = (Relacionamento_Escol_Dep_Bons.shape[0]/
(Relacionamento_Escol_Dep_Bons.shape[0] + Relacionamento_Escol_Dep_Mals.shape[0])) * 100

print('Relacionamento entre Escolaridade: Ensino Médio | COM Dependentes\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_Escol_Dep_Bons, 
                                                     (100 - porcentagem_Escol_Dep_Bons)))


Quantidade de dados: 6693

Relacionamento entre Escolaridade: Ensino Médio | COM Dependentes

Bons pagadores: 92.23068877932168%
Mals pagadores: 7.769311220678318%


In [62]:
Relacionamento_chequeSemFundo_VR_Multa_Bons = bons_pagadores.query(
    '`Cheque Sem Fundo` == "S" & `Valor Real Pago %` <= 60 & `Multa %` <= 10' )

Relacionamento_chequeSemFundo_VR_Multa_Mals = mals_pagadores.query(
    '`Cheque Sem Fundo` == "S" & `Valor Real Pago %` <= 60 & `Multa %` <= 10' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_chequeSemFundo_VR_Multa_Bons.shape[0] + 
 Relacionamento_chequeSemFundo_VR_Multa_Mals.shape[0]))


porcentagem_chequeSemFundo_VR_Multa_Bons = (Relacionamento_chequeSemFundo_VR_Multa_Bons.shape[0]/
(Relacionamento_chequeSemFundo_VR_Multa_Bons.shape[0] + 
 Relacionamento_chequeSemFundo_VR_Multa_Mals.shape[0])) * 100

print('Relacionamento entre COM Cheque Sem Fundo | Valor Real <=60% | Multa <= 20%\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_chequeSemFundo_VR_Multa_Bons, 
                                                     (100 - porcentagem_chequeSemFundo_VR_Multa_Bons)))


Quantidade de dados: 5713

Relacionamento entre COM Cheque Sem Fundo | Valor Real <=60% | Multa <= 20%

Bons pagadores: 96.09662173989148%
Mals pagadores: 3.903378260108525%


In [61]:
Relacionamento_VR_Multa_Bons = bons_pagadores.query(
    '`Valor Real Pago %` <= 60 & `Multa %` <= 10' )

Relacionamento_VR_Multa_Mals = mals_pagadores.query(
    '`Valor Real Pago %` <= 60 & `Multa %` <= 10' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_VR_Multa_Bons.shape[0] + 
 Relacionamento_VR_Multa_Mals.shape[0]))


porcentagem_VR_Multa_Bons = (Relacionamento_VR_Multa_Bons.shape[0]/
(Relacionamento_VR_Multa_Bons.shape[0] + 
 Relacionamento_VR_Multa_Mals.shape[0])) * 100

print('Relacionamento entre Valor Real <=60% | Multa <= 10%\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_VR_Multa_Bons, 
                                                     (100 - porcentagem_VR_Multa_Bons)))

Quantidade de dados: 19841

Relacionamento entre Valor Real <=60% | Multa <= 10%

Bons pagadores: 94.1636006249685%
Mals pagadores: 5.836399375031505%


In [50]:
Relacionamento_EC_Multa_Bons = bons_pagadores.query(
    '`Estado Civil` == 2 & `Multa %` <= 10' )

Relacionamento_EC_Multa_Mals = mals_pagadores.query(
    '`Estado Civil` == 2 & `Multa %` <= 10' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_EC_Multa_Bons.shape[0] + 
 Relacionamento_EC_Multa_Mals.shape[0]))


porcentagem_EC_Multa_Bons = (Relacionamento_EC_Multa_Bons.shape[0]/
(Relacionamento_EC_Multa_Bons.shape[0] + 
 Relacionamento_EC_Multa_Mals.shape[0])) * 100

print('Relacionamento entre Estado Civil: Solteiro | Multa <= 10%\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_EC_Multa_Bons, 
                                                     (100 - porcentagem_EC_Multa_Bons)))

Quantidade de dados: 14835

Relacionamento entre Estado Civil: Solteiro | Multa <= 10%

Bons pagadores: 95.18705763397371%
Mals pagadores: 4.8129423660262916%


In [55]:
Relacionamento_Escol_Multa_Bons = bons_pagadores.query(
    'Escolaridade == 1 & `Multa %` <= 10' )

Relacionamento_Escol_Multa_Mals = mals_pagadores.query(
    'Escolaridade == 1 & `Multa %` <= 10' )

#Print quantidade de dados filtrados
print("Quantidade de dados: {}\n".format(Relacionamento_Escol_Multa_Bons.shape[0] + 
 Relacionamento_Escol_Multa_Mals.shape[0]))


porcentagem_Escol_Multa_Bons = (Relacionamento_Escol_Multa_Bons.shape[0]/
(Relacionamento_Escol_Multa_Bons.shape[0] + 
 Relacionamento_Escol_Multa_Mals.shape[0])) * 100

print('Relacionamento entre Escolaridade: Ensino Médio | Multa <= 10%\n\n' +
        'Bons pagadores: {}%\nMals pagadores: {}%'.format(porcentagem_Escol_Multa_Bons, 
                                                     (100 - porcentagem_Escol_Multa_Bons)))

Quantidade de dados: 6702

Relacionamento entre Escolaridade: Ensino Médio | Multa <= 10%

Bons pagadores: 95.29991047448523%
Mals pagadores: 4.70008952551477%
